# Fine tuning FLAN-T5 on Drug Discovery

- Author: [Khairi Abidi](https://github.com/abidikhairi)
- Author: [Ranim Mechergui](https://github.com/ranim.mechergui)

This notebook demonstrates finetuning of FLAN-T5 for drug discovery instruction.

Key Features:

- Memory Efficient: LoRA for consumer GPUs

The model learns to recommend adequate drugs based on gene context.

In [1]:
%env CUDA_VISIBLE_DEVICES=0,1

env: CUDA_VISIBLE_DEVICES=0,1


In [2]:
%env WANDB_PROJECT=Drug-FLAN

env: WANDB_PROJECT=Drug-FLAN


## Connect to 3rd party services

- **WandB**: for experiment tracking.
- **HuggingFace Hub**: for model checkpoints uploading.

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HUGGING_FACE_TOKEN")
wandb_token = user_secrets.get_secret("WANDB_API_KEY")

In [4]:
!wandb login {wandb_token}

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [5]:
!huggingface-cli login --token {hf_token}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `KAGGLE_TOKEN` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `KAGGLE_TOKEN`


## GPU Environment Detection
Verify GPU availability and display hardware specifications for optimal training configuration.

In [6]:
import torch

# Verify CUDA availability and display GPU specifications
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Number of GPUs: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    # Display current GPU details for training optimization
    print(f"Current GPU: {torch.cuda.current_device()}")
    print(f"GPU name: {torch.cuda.get_device_name()}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    # Provide guidance for enabling GPU in Colab
    print("⚠️  No GPU available. This notebook requires a GPU for efficient training.")
    print("In Colab: Runtime → Change runtime type → Hardware accelerator → GPU")

CUDA available: True
Number of GPUs: 2
Current GPU: 0
GPU name: Tesla T4
GPU memory: 15.8 GB


## Core Library Imports
Import essential libraries for finetuning, model configuration, and experiment tracking.

In [7]:
# Model and tokenization
from transformers import (
    T5ForConditionalGeneration,    # Seq2Seq language model loading
    T5Tokenizer,                   # Text tokenization
    DataCollatorForSeq2Seq,        # Batch inputs handling
)

# Model optimization
from torch.optim import AdamW
from transformers import get_scheduler

# Training and Setup
from transformers import (
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

# Dataset handling
from datasets import load_dataset

# Logging configuration
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

2025-10-08 15:40:50.550468: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759938050.762161      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759938050.820460      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
model_name = "google/flan-t5-base"
max_seq_length = 512

print(f"Loading model: {model_name}")
print(f"Max sequence length: {max_seq_length}")

Loading model: google/flan-t5-base
Max sequence length: 512


In [9]:
model = T5ForConditionalGeneration.from_pretrained(
    model_name,
    trust_remote_code=True,               # Allow custom model code execution
    # dtype=torch.float16,                  # Use FP16 for non-quantized operations
)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
tokenizer = T5Tokenizer.from_pretrained(model_name, trust_remote_code=True)

# Ensure tokenizer has proper padding token for batch processing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [11]:
print(f"✅ Model loaded successfully!")
print(f"📊 Model parameters: ~{sum(p.numel() for p in model.parameters()) / 1e6:.1f}M")
print(f"🧮 Quantized parameters: ~{sum(p.numel() for p in model.parameters() if hasattr(p, 'quant_type')) / 1e6:.1f}M")

✅ Model loaded successfully!
📊 Model parameters: ~247.6M
🧮 Quantized parameters: ~0.0M


In [12]:
def compute_model_size(model):
    n_params = 0
    for p in model.parameters():
        n_params += p.nelement() * p.element_size()
    for p in model.buffers():
        n_params += p.nelement() * p.element_size()

    return n_params / (1024 ** 3)

print(f"📊 Model size : {compute_model_size(model):.2f} GB")

📊 Model size : 0.92 GB


## Dataset Setup
Configure the Drug discovery instruction dataset.

In [13]:
def tokenize_dataset_example(examples):
    model_inputs = tokenizer(examples['input'], return_tensors='pt', padding=True)
    labels = tokenizer(examples['target'], return_tensors='pt', padding=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

print("✅ Dataset tokenization function defined (batch mode)")

✅ Dataset tokenization function defined (batch mode)


In [14]:
def filter_dataset_example(example):
    return len(example['input_ids']) < 512
    
print("✅ Dataset filter functions defined")

✅ Dataset filter functions defined


In [15]:
# Load and preprocess DrugInstruct training dataset
print("🔄 Loading DrugInstruct dataset...")
dataset = load_dataset("khairi/drug-discovery-hetionet")

# Apply conversation formatting to all examples
dataset = dataset.map(tokenize_dataset_example, batched=True, batch_size=4)
    # .filter(filter_dataset_example)

train_data = dataset['train']
valid_data = dataset['validation']

print(f"✅ Dataset loaded and processed!")
print(f"📊 Training examples: {len(train_data):,}")
print(f"📊 Validation examples: {len(valid_data):,}")
print(f"🎯 Sample input: {tokenizer.decode(train_data[0]['input_ids'])}")
print(f"🎯 Sample output: {tokenizer.decode(train_data[0]['labels'])}")

🔄 Loading DrugInstruct dataset...


train.parquet:   0%|          | 0.00/164k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/15.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1416 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/84 [00:00<?, ? examples/s]

Map:   0%|          | 0/1416 [00:00<?, ? examples/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

✅ Dataset loaded and processed!
📊 Training examples: 1,416
📊 Validation examples: 84
🎯 Sample input: Predict drugs for the central gene based on its neighborhood: Central node: NAT2 One-hop neighbors: [process, Biological Process::GO:0071466], [process, Biological Process::GO:0009410], [process, Biological Process::GO:0006805] [molecular function, Molecular Function::GO:0004060], [molecular function, Molecular Function::GO:0008080], [molecular function, Molecular Function::GO:0016746], [molecular function, Molecular Function::GO:0016407], [molecular function, Molecular Function::GO:0016747] Answer:</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

## Training Setup
Configure training parameters optimized for finetuning with memory constraints.

In [16]:
train_data

Dataset({
    features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1416
})

In [17]:
valid_data

Dataset({
    features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 84
})

In [18]:
# Prepare data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [19]:
# Configure Finetuning training parameters for sequence modeling
training_args = Seq2SeqTrainingArguments(
    # Memory-efficient batch configuration
    per_device_train_batch_size=4,   # Small batch for GPU memory constraints
    gradient_accumulation_steps=4,   # Effective batch size = 4 * 4 = 16
       
    # Training duration and monitoring
    # max_steps=1000,                     # Short demo run (increase to 500+ for production)
    logging_steps=50,                  # Log metrics every step for close monitoring
    save_steps=50,
    eval_steps=50,
    eval_strategy='steps',
    num_train_epochs=50,            # Uncomment for production

    weight_decay=0.01,
    optim = 'adamw_torch',
    lr_scheduler_type='cosine',
    warmup_ratio=0.2,
    learning_rate=1e-4,
    save_total_limit = 3,
    
    # Stability and output configuration
    output_dir="./finetuning_outputs",
    max_grad_norm=0.1,               # Aggressive gradient clipping for stable training
    report_to="wandb",                # use wandb for experiment tracking
    run_name='flan-base-hetionet-instruct',

    # Push to Hub, uncomment in production
    push_to_hub=True,
    hub_model_id='khairi/Shizuku-247M'
    
)

In [20]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=valid_data,
    tokenizer=tokenizer,
    data_collator=data_collator
)

/tmp/ipykernel_19/1933226986.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [21]:
# Execute FineTuning
print("🚀 Starting Finetuning...")

# Run the training process
trainer.train()

print("✅ Training completed successfully!")
print(f"💾 Model saved to: {training_args.output_dir}")

wandb: Currently logged in as: flursky to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


🚀 Starting Finetuning...


wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20251008_154121-ba9wcdqp
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run flan-base-hetionet-instruct
wandb: ⭐️ View project at https://wandb.ai/flursky/Drug-FLAN
wandb: 🚀 View run at https://wandb.ai/flursky/Drug-FLAN/runs/ba9wcdqp
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss
50,21.304500,16.307005
100,9.889800,3.979970
150,3.098500,1.237166
200,1.511900,0.902283
250,1.042200,0.758416
300,0.876100,0.690436
350,0.783100,0.647113
400,0.726700,0.604196
450,0.654700,0.586449
500,0.608000,0.567334


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

✅ Training completed successfully!
💾 Model saved to: ./finetuning_outputs


In [22]:
row = valid_data[0]

In [23]:
row.keys()

dict_keys(['input', 'target', 'input_ids', 'attention_mask', 'labels'])

In [24]:
inputs = tokenizer(row['input'], return_tensors='pt')

inputs = {k: v.to(model.device) for k, v in inputs.items()}

In [25]:
output = model.generate(**inputs)

In [26]:
print(tokenizer.decode(output[0]))

<pad> - L-Aspartic Acid</s>


In [27]:
print(row['target'])

- Adenosine monophosphate 
- Adenosine triphosphate
